# Задание по программированию: 1NN против RandomForest
В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test. Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.  

Задание 1 - Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.  
Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).  
Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.
Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np

digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=False)

X.shape, X_train.shape, X_test.shape

((1797, 64), (1347, 64), (450, 64))

In [2]:
# ver1 НАДО РАЗОБРАТЬСЯ НО Я УСТАЛ!
def determine_knn(x, X, y):
    distances = ((X - x) ** 2).sum(axis = 1)
    min_d = distances[0]
    min_t = y[0]
    for d, t in zip(distances, y):
        if d < min_d:
            min_d = d
            min_t = t
    return min_t

predictions = []
for i in range(X_test.shape[0]):
    x = X_test[i, :]
    predictions.append(determine_knn(x, X_train, y_train))

predictions = np.array(predictions)
accuracy = float((y_test != predictions).sum())/y_test.shape[0]
accuracy

0.03777777777777778

In [3]:
# ver2 НАДО РАЗОБРАТЬСЯ НО Я УСТАЛ!
def nnclassify(X_test, X_train, y_train):
    res = np.array([])
    for i in range(len(X_test)):
        dist = np.array([])
        a = np.array(X_test[i])
        for j in range(len(X_train)):
            b = np.array(X_train[j])
            dist = np.append(dist, np.linalg.norm(a-b, ord=2))
        res = np.append(res, int(y_train[np.argmin(dist)]))
    return res
y_pred = nnclassify(X_test, X_train, y_train)
print sum([y_test[i] != y_pred[i] for i in range(len(y_test))])/float(len(y_test))

0.03777777777777778


Задание 2 - Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [8]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=1000)
RFC.fit(X_train, y_train)
pred = RFC.predict(X_test)
acc = float(sum(pred != y_test)) / len(y_test)
print acc

0.06444444444444444